In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Cargar base SIVIGILA346

In [ ]:
df_sivigila346 = pd.read_csv("../Github/dl-covid19-descriptive-reports/MinSalud/sivigila346/data/ExtraccionEv346.txt",
                             low_memory=False,
                             on_bad_lines='skip',
                             sep='|',
                             usecols = ['PersonaBasicaID',
                                        'FechaNotificacion',
                                        'FechaInicioSintomas', 
                                        'FechaDefuncion', 
                                        'DepartamentoNotificacion'])

print('Número filas y columnas: ', df_sivigila346.shape)

In [ ]:
# Eliminar registros basura (son 2)
df_sivigila346 = df_sivigila346.dropna(subset=['FechaInicioSintomas'])
print('Número filas y columnas: ', df_sivigila346.shape)

## Análisis para Bogotá

In [ ]:
# Seleccionar los datos de Bogotá
df_bog = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] == 'Bogotá, D.C.'].copy()

In [ ]:
# Convertir las fechas a formato 'datetime'
df_bog['FechaInicioSintomas'] = pd.to_datetime(df_bog['FechaInicioSintomas'].astype(int), format='%Y%m%d')
df_bog['FechaDefuncion'] = pd.to_datetime(df_bog['FechaDefuncion'].astype(int), format='%Y%m%d')
df_bog['FechaNotificacion'] = pd.to_datetime(df_bog['FechaNotificacion'].astype(int), format='%Y%m%d')

In [ ]:
# Eliminar fechas de defunción erróneas
# Al mismo tiempo, se selecciona la gente que fallecio
df_bog = df_bog[df_bog['FechaDefuncion'] != '19000101']

In [ ]:
# ============================================
# Descartar los casos donde el ID es igual a 1
# ============================================
df_bog = df_bog[df_bog['PersonaBasicaID'] != '1']

In [ ]:
# Cálculo de 'onset_death' en días
df_bog['Onset_death'] = (df_bog['FechaDefuncion'] - df_bog['FechaInicioSintomas']).dt.days
df_bog.head()

In [ ]:
# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

# Calcular promedio de 'Onset_death' por olas
promedios = []
for nombre, (inicio, fin) in waves.items():
    filtro = (df_bog['FechaNotificacion'] >= inicio) & (df_bog['FechaNotificacion'] <= fin)
    promedio = df_bog.loc[filtro, 'Onset_death'].mean()
    promedios.append(promedio)
    print(nombre)
    print('Promedio: ', promedio)
    print("# Registros: ", len(df_bog.loc[filtro, 'Onset_death']))
    print("")

In [ ]:
# Grafica 
plt.figure(figsize=(4, 5))
plt.bar(waves.keys(), promedios, color=['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c'], width=1.0)
# plt.xlabel('COVID-19 waves')
plt.ylabel('Avg value of delay time (Days)', fontsize=14)
plt.title('Onset to death Bogotá')
plt.yticks([0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0, 22.5, 25])

for i, promedio in enumerate(promedios):
    plt.text(i, promedio + 0.2, f'{promedio:.2f}', ha='center', fontsize=9)

plt.show()

## Análisis Nacional

In [ ]:
# Seleccionar los datos Colombia
df_col = df_sivigila346.copy()

In [ ]:
# Convertir las fechas a formato 'datetime'
df_col['FechaInicioSintomas'] = pd.to_datetime(df_col['FechaInicioSintomas'].astype(int), format='%Y%m%d')
df_col['FechaDefuncion'] = pd.to_datetime(df_col['FechaDefuncion'].astype(int), format='%Y%m%d')
df_col['FechaNotificacion'] = pd.to_datetime(df_col['FechaNotificacion'].astype(int), format='%Y%m%d')

In [ ]:
# Eliminar fechas de defunción erróneas
df_col = df_col[df_col['FechaDefuncion'] != '19000101']

In [ ]:
# ============================================
# Descartar los casos donde el ID es igual a 1
# ============================================
df_col = df_col[df_col['PersonaBasicaID'] != '1']

In [ ]:
# Cálculo de 'Onset_death' en días
df_col['Onset_death'] = (df_col['FechaDefuncion'] - df_col['FechaInicioSintomas']).dt.days
df_col.head()

In [ ]:
# Definir olas COVID-19 (obtenidas de: https://github.com/TRACE-LAC/covid19-waves-bogota/blob/main/waves/outputs/waves.csv)
waves = {
    'Ola 1': ('2020-02-26', '2020-09-25'),
    'Ola 2': ('2020-11-01', '2021-03-01'),
    'Ola 3': ('2021-03-01', '2021-09-14'),
    'Ola 4': ('2021-11-20', '2022-03-24')
}

# Calcular promedio de 'Onset_death' por olas
promedios = []
for nombre, (inicio, fin) in waves.items():
    filtro = (df_col['FechaNotificacion'] >= inicio) & (df_col['FechaNotificacion'] <= fin)
    promedio = df_col.loc[filtro, 'Onset_death'].mean()
    promedios.append(promedio)
    print(nombre)
    print('Promedio: ', promedio)
    print("# Registros: ", len(df_col.loc[filtro, 'Onset_death']))
    print("")

In [ ]:
# Grafica 
plt.figure(figsize=(6, 4))
plt.bar(waves.keys(), promedios, color=['#6a5acd', '#66c2a5', '#4daf4a', '#d95f02'], width=0.7, zorder=3)
# plt.xlabel('COVID-19 waves')
plt.ylabel('Promedio (días)', fontsize=12)
plt.title('Inicio de síntomas hasta fallecimiento', fontsize=14)
plt.yticks([0, 5, 10, 15, 20, 25], fontsize=10)

for i, promedio in enumerate(promedios):
    plt.text(i, promedio + 0.2, f'{promedio:.1f}', ha='center', fontsize=10)

plt.grid(True, linestyle='--', alpha=0.6, zorder=0)
plt.show()

In [ ]:
for nombre, (inicio, fin) in waves.items():
    filtro = (df_col['FechaNotificacion'] >= inicio) & (df_col['FechaNotificacion'] <= fin)
    print(nombre, ": ", len(df_col.loc[filtro, 'Onset_death']))
#     print(df_col[filtro])

---
---
---
## Análisis para todos los departamentos

In [ ]:
df_sivigila346_v2 = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] != 'NO REPORTADO'].copy()

In [ ]:
# https://dgn.isolutions.iso.org/obp/ui#iso:code:3166:CO
df_sivigila346_v2['DepartamentoNotificacion'].replace({'Bogotá, D.C.': 'DC', 
                                                       'Antioquia': 'ANT', 
                                                       'Valle del Cauca': 'VAC', 
                                                       'Cundinamarca': 'CUN', 
                                                       'Santander': 'SAN', 
                                                       'Atlántico': 'ATL', 
                                                       'Boyacá': 'BOY', 
                                                       'Norte de Santander': 'NSA', 
                                                       'Córdoba': 'COR', 
                                                       'Tolima': 'TOL', 
                                                       'Caldas': 'CAL', 
                                                       'Cesar': 'CES', 
                                                       'Meta': 'MET', 
                                                       'Risaralda': 'RIS', 
                                                       'Huila': 'HUI', 
                                                       'Bolívar': 'BOL', 
                                                       'Cauca': 'CAU', 
                                                       'Sucre': 'SUC', 
                                                       'Quindio': 'QUI', 
                                                       'Nariño': 'NAR', 
                                                       'Magdalena': 'MAG', 
                                                       'Casanare': 'CAS', 
                                                       'La Guajira': 'LAG', 
                                                       'Caquetá': 'CAQ', 
                                                       'Putumayo': 'PUT', 
                                                       'Chocó': 'CHO', 
                                                       'Arauca': 'ARA', 
                                                       'Archipiélago de San Andrés, Providencia y Santa Catalina': 'SAP', 
                                                       'Amazonas': 'AMA', 
                                                       'Guaviare': 'GUV', 
                                                       'Guainía': 'GUA', 
                                                       'Vichada': 'VIC', 
                                                       'Vaupés': 'VAU'}, inplace=True)

In [ ]:
# Definir las olas de COVID-19
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

# Inicializar lista para guardar los resultados
results = []
conteos = []

# Iterar por cada departamento
for departamento in df_sivigila346_v2['DepartamentoNotificacion'].unique():
    # Seleccionar los datos por departamento
    df_dep = df_sivigila346_v2[df_sivigila346_v2['DepartamentoNotificacion'] == departamento].copy()
    
    # Convertir las fechas a formato 'datetime'
    df_dep['FechaInicioSintomas'] = pd.to_datetime(df_dep['FechaInicioSintomas'].astype(int), format='%Y%m%d')
    df_dep['FechaDefuncion'] = pd.to_datetime(df_dep['FechaDefuncion'].astype(int), format='%Y%m%d')
    df_dep['FechaNotificacion'] = pd.to_datetime(df_dep['FechaNotificacion'].astype(int), format='%Y%m%d')
    
    # Eliminar fechas de defunción erróneas
    df_dep = df_dep[df_dep['FechaDefuncion'] != '19000101']
    
    # ============================================
    # Descartar los casos donde el ID es igual a 1
    # ============================================
    df_dep = df_dep[df_dep['PersonaBasicaID'] != '1']
    
    # Cálculo de 'Onset_death' en días
    df_dep['Onset_death'] = (df_dep['FechaDefuncion'] - df_dep['FechaInicioSintomas']).dt.days
    
    # Calcular promedio de 'Onset_death' por olas
    for wave, (inicio, fin) in waves.items():
        filtro = (df_dep['FechaNotificacion'] >= inicio) & (df_dep['FechaNotificacion'] <= fin)
        promedio = df_dep.loc[filtro, 'Onset_death'].mean()
        results.append({
            'Departamento': departamento,
            'Wave': wave,
            'MeanDelay': promedio
        })
        conteos.append({
            'Departamento': departamento,
            'Wave': wave,
            'Registros': len(df_dep.loc[filtro, 'Onset_death'])
        })

In [ ]:
# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)
pivot_table = df_results.pivot(index="Departamento", columns="Wave", values="MeanDelay")

df_conteos = pd.DataFrame(conteos)
tabla_conteos = df_conteos.pivot(index="Departamento", columns="Wave", values="Registros")

In [ ]:
tabla_conteos

In [ ]:
# Grafica heatmap
plt.figure(figsize=(3, len(pivot_table)*0.25)) 
sns.heatmap(pivot_table, 
            annot=True, 
            cmap="coolwarm", #RdYlGr coolwarm summer
            fmt=".2f",
            annot_kws={"size": 8})
plt.title("Avg Delay from\n Onset to Death", fontsize=12)
plt.xlabel("")
plt.xticks(rotation=60)
plt.ylabel("Department")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 5))
sns.boxplot(x='Wave', y='MeanDelay', data=df_results, palette="Set3", hue='Wave', legend=False, showfliers=False)
sns.swarmplot(x='Wave', y='MeanDelay', data=df_results, color=".25")
plt.title("Onset to Death")
plt.xlabel("")
plt.ylabel("Avg value of delay time (Days)")
plt.tight_layout()
plt.show()

---
---
---
## Análisis para todos los departamentos POR REGIONES

In [ ]:
df_sivigila346_v2 = df_sivigila346[df_sivigila346['DepartamentoNotificacion'] != 'NO REPORTADO'].copy()

In [ ]:
# https://dgn.isolutions.iso.org/obp/ui#iso:code:3166:CO
df_sivigila346_v2['DepartamentoNotificacion'].replace({'Bogotá, D.C.': 'DC', 
                                                       'Antioquia': 'ANT', 
                                                       'Valle del Cauca': 'VAC', 
                                                       'Cundinamarca': 'CUN', 
                                                       'Santander': 'SAN', 
                                                       'Atlántico': 'ATL', 
                                                       'Boyacá': 'BOY', 
                                                       'Norte de Santander': 'NSA', 
                                                       'Córdoba': 'COR', 
                                                       'Tolima': 'TOL', 
                                                       'Caldas': 'CAL', 
                                                       'Cesar': 'CES', 
                                                       'Meta': 'MET', 
                                                       'Risaralda': 'RIS', 
                                                       'Huila': 'HUI', 
                                                       'Bolívar': 'BOL', 
                                                       'Cauca': 'CAU', 
                                                       'Sucre': 'SUC', 
                                                       'Quindio': 'QUI', 
                                                       'Nariño': 'NAR', 
                                                       'Magdalena': 'MAG', 
                                                       'Casanare': 'CAS', 
                                                       'La Guajira': 'LAG', 
                                                       'Caquetá': 'CAQ', 
                                                       'Putumayo': 'PUT', 
                                                       'Chocó': 'CHO', 
                                                       'Arauca': 'ARA', 
                                                       'Archipiélago de San Andrés, Providencia y Santa Catalina': 'SAP', 
                                                       'Amazonas': 'AMA', 
                                                       'Guaviare': 'GUV', 
                                                       'Guainía': 'GUA', 
                                                       'Vichada': 'VIC', 
                                                       'Vaupés': 'VAU'}, inplace=True)

In [ ]:
def classify_region(department):
    region_map = {
        "Amazónica": ["AMA", "CAQ", "GUA", "GUV", "PUT", "VAU"],
        "Andina": ["ANT", "BOY", "CAL", "CUN", "DC", "HUI", "NSA", "QUI", "RIS", "SAN", "TOL"],
        "Pacífica": ["VAC", "CHO", "CAU", "NAR"],
        "Caribe e Insular": ["ATL", "BOL", "CES", "COR", "LAG", "MAG", "SUC", "SAP"],
        "Orinoquía": ["ARA", "CAS", "MET", "VIC"]
    }
    
    for region, departments in region_map.items():
        if department in departments:
            return region
    return "Desconocido"  # En caso de que haya códigos no contemplados

# Aplicar la función a la columna DepartamentoNotificacion
df_sivigila346_v2["Region"] = df_sivigila346_v2["DepartamentoNotificacion"].apply(classify_region)

In [ ]:
# Definir las olas de COVID-19
waves = {
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

# Inicializar lista para guardar los resultados
results = []
conteos = []

# Iterar por cada departamento
for region in df_sivigila346_v2['Region'].unique():
    # Seleccionar los datos por departamento
    df_reg = df_sivigila346_v2[df_sivigila346_v2['Region'] == region].copy()
    
    # Convertir las fechas a formato 'datetime'
    df_reg['FechaInicioSintomas'] = pd.to_datetime(df_reg['FechaInicioSintomas'].astype(int), format='%Y%m%d')
    df_reg['FechaDefuncion'] = pd.to_datetime(df_reg['FechaDefuncion'].astype(int), format='%Y%m%d')
    df_reg['FechaNotificacion'] = pd.to_datetime(df_reg['FechaNotificacion'].astype(int), format='%Y%m%d')
    
    # Eliminar fechas de defunción erróneas
    df_reg = df_reg[df_reg['FechaDefuncion'] != '19000101']
    
    # ============================================
    # Descartar los casos donde el ID es igual a 1
    # ============================================
    df_reg = df_reg[df_reg['PersonaBasicaID'] != '1']
    
    # Cálculo de 'Onset_death' en días
    df_reg['Onset_death'] = (df_reg['FechaDefuncion'] - df_reg['FechaInicioSintomas']).dt.days
    
    # Calcular promedio de 'Onset_death' por olas
    for wave, (inicio, fin) in waves.items():
        filtro = (df_reg['FechaNotificacion'] >= inicio) & (df_reg['FechaNotificacion'] <= fin)
        promedio = df_reg.loc[filtro, 'Onset_death'].mean()
        results.append({
            'Region': region,
            'Wave': wave,
            'MeanDelay': promedio
        })
        conteos.append({
            'Region': region,
            'Wave': wave,
            'Registros': len(df_reg.loc[filtro, 'Onset_death'])
        })

In [ ]:
# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)
pivot_table = df_results.pivot(index="Region", columns="Wave", values="MeanDelay")

df_conteos = pd.DataFrame(conteos)
tabla_conteos = df_conteos.pivot(index="Region", columns="Wave", values="Registros")

In [ ]:
tabla_conteos

In [ ]:
# Gráfica
plt.figure(figsize=(4, 3.5)) 
sns.heatmap(pivot_table, 
            annot=True, 
            cmap="coolwarm", #RdYlGr coolwarm summer
            fmt=".2f",
            annot_kws={"size": 9})
plt.title("Avg Delay from\n Onset to Death", fontsize=12)
plt.xticks(rotation=45)
plt.xlabel("")
plt.ylabel("", fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))

# Crear líneas para cada departamento
for reg in pivot_table.index:
    plt.plot(pivot_table.columns, pivot_table.loc[reg], marker='o', linestyle='-', label=reg)

plt.title("Onset to Death\nby Region & Wave", fontsize=14)
plt.xlabel("")
plt.ylabel('Avg value of delay time (Days)', fontsize=12)
plt.legend(title="Region", loc='upper right', fontsize=9, bbox_to_anchor=(1.34, 1.02)) 
plt.grid(True, linestyle='--', alpha=0.7)

plt.show()

In [ ]:
plt.figure(figsize=(8, 4))

num_regions = len(pivot_table)
num_waves = len(pivot_table.columns)

x = np.arange(num_regions)
width = 0.15  

# Barras
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c']
for i, wave in enumerate(pivot_table.columns):
    plt.bar(x + i * width, pivot_table[wave], width=width, label=wave, color=colors[i])

plt.xticks(x + width * (num_waves / 2 - 0.5), pivot_table.index, rotation=0, fontsize=9)
plt.ylabel('Avg value of delay time (Days)')
plt.title("Onset to Death")
plt.legend(title="COVID-19 Wave", bbox_to_anchor=(1.25, 1.02))
plt.yticks([0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20.0, 22.5, 25.0, 27.5])

plt.tight_layout()
plt.show()